In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import directory_functions as dirfuncs
import gather_middleware_statistics as gmws
import gather_memtier_statistics as gmts
import cut_away_warmup_cooldown as cut
import math
exp_dir = "/home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817"

In [2]:
# Get response times from MW for 6 multigets
def gather_requests(exp_dir, reps, middlewares):
    all_reps = []
    for rep in reps:

        middleware_dirs = [dirfuncs.get_only_subdir(os.path.join(exp_dir, str(rep), mw_dir)) for mw_dir in middlewares]
        concatenated_requests = [gmws.concatenate_requestlogs(middleware_dir) for middleware_dir in middleware_dirs]


        
        metrics = [gmws.extract_metrics(reqs) for reqs in concatenated_requests]


        cut_metrics = [cut.cut_away_warmup_cooldown(mets, 10, 72) for mets in metrics]

        all_reps.extend(cut_metrics)
        
    return pd.concat(all_reps)


In [3]:
num_workers = 8
run = "writeOnly_64vc{}workers".format(num_workers)
inputdir = os.path.join(exp_dir, run)
middlewares = ["middleware_04", "middleware_05"]
                        
reps_range = range(1, 4)
reqs_mw1 = gather_requests(inputdir, reps_range, ["middleware_04"])
reqs_mw2 = gather_requests(inputdir, reps_range, ["middleware_05"])


Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/middleware_04/2017-11-27 220626
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/2/middleware_04/2017-11-27 230115
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/3/middleware_04/2017-11-27 235558
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/middleware_05/2017-11-27 220626
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/2/middleware_05/2017-11-27 230115
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOn

In [4]:
ping_log_filenames_c1 = ["client_ping_01_0.log", "client_ping_01_1.log"]
ping_log_filenames_c2 = ["client_ping_02_0.log", "client_ping_02_1.log"]

all_rep_ping_logs_c1 = []
all_rep_ping_logs_c2 = []
for rep in reps_range:
    rep_dir = os.path.join(inputdir, str(rep))
    ping_log_filepaths_c1 = [os.path.join(rep_dir, logfile) for logfile in ping_log_filenames_c1]
    ping_log_filepaths_c2 = [os.path.join(rep_dir, logfile) for logfile in ping_log_filenames_c2]
    ping_logs_c1 = [gmts.extract_ping_logs(filepath) for filepath in ping_log_filepaths_c1]
    ping_logs_c2 = [gmts.extract_ping_logs(filepath) for filepath in ping_log_filepaths_c2]
    all_rep_ping_logs_c1.extend(ping_logs_c1)
    all_rep_ping_logs_c2.extend(ping_logs_c2)

avg_rtt_c1 = pd.concat(all_rep_ping_logs_c1)['rtt'].mean()
avg_rtt_c2 = pd.concat(all_rep_ping_logs_c2)['rtt'].mean()

Extracting stats from ping log /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/client_ping_01_0.log
Extracting stats from ping log /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/client_ping_01_1.log
Extracting stats from ping log /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/client_ping_02_0.log
Extracting stats from ping log /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/client_ping_02_1.log
Extracting stats from ping log /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/2/client_ping_01_0.log
Extracting stats from ping log /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/2/client_ping_01_1.log
Extr

In [5]:
client_logfiles = ["client_01_0.log", "client_01_1.log", "client_02_0.log", "client_02_1.log"]

all_reps = []
for rep in reps_range:

    log_folder_path = os.path.join(inputdir, str(int(rep)))

    # Now we extract throughput, responsetime, average queuetime and missrate from the middleware
    middleware_dirs = [dirfuncs.get_only_subdir(os.path.join(log_folder_path, mw_dir)) for mw_dir in middlewares]
    concatenated_requests = [gmws.concatenate_requestlogs(middleware_dir) for middleware_dir in middleware_dirs]
    metrics = [gmws.extract_metrics(reqs) for reqs in concatenated_requests]
    cut_metrics = [cut.cut_away_warmup_cooldown(mets, 10, 72) for mets in metrics]
    windows = [gmws.aggregate_over_windows(cut_mets) for cut_mets in cut_metrics]
    rep_metrics = gmws.aggregate_over_middlewares(windows)
    all_reps.append(rep_metrics)

mw_agg_over_reps = gmws.aggregate_over_reps(all_reps)
mw_averages = gmws.aggregate_over_timesteps(mw_agg_over_reps)
avg = mw_averages.loc['mean', :]

all_rep_windows = []
for rep in reps_range:

    log_folder_path = os.path.join(inputdir, str(rep))

    client_logfile_paths = [os.path.join(log_folder_path, client_logfile) for client_logfile in client_logfiles]
    window = gmts.aggregate_over_clients(client_logfile_paths)
    window = cut.cut_away_warmup_cooldown(window, 10, 72)
    all_rep_windows.append(window)

mt_agg_over_reps = gmts.aggregate_over_reps(all_rep_windows)
mt_averages = gmts.aggregate_over_timesteps(mt_agg_over_reps)

Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/middleware_04/2017-11-27 220626
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/1/middleware_05/2017-11-27 220626
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/2/middleware_04/2017-11-27 230115
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/2/middleware_05/2017-11-27 230115
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOnly_64vc8workers/3/middleware_04/2017-11-27 235558
Concatenating requestlogs in /home/flo/Documents/eth-asl-final-experiment-data/exp3/3_2_rerun_twomt_2017-11-27_213817/writeOn

In [6]:
print("Service times input for simple model:")
print("1: {:0.4f}".format(avg_rtt_c1/2))
print("2: {:0.4f}".format(avg_rtt_c2/2))
print("3: {:0.4f}".format(reqs_mw1['netthreadServiceTime_ms'].mean()))
print("4: {:0.4f}".format(reqs_mw2['netthreadServiceTime_ms'].mean()))
print("5: {:0.4f}".format(reqs_mw1['workerServiceTime_ms'].mean()))
print("6: {:0.4f}".format(reqs_mw2['workerServiceTime_ms'].mean()))
print("7: {:0.4f}".format(avg_rtt_c1/2))
print("8: {:0.4f}".format(avg_rtt_c2/2))

print("\n\n")

xput_mw1 = reqs_mw1['responseTime_ms'].count() / (reqs_mw1.index.max() - reqs_mw1.index.min()) * 20 / 3
xput_mw2 = reqs_mw2['responseTime_ms'].count() / (reqs_mw2.index.max() - reqs_mw2.index.min()) * 20 / 3
print("Actually measured metrics: (compare with model output)")
print("Throughput MW1: {:0.2f}".format(xput_mw1))
print("Throughput MW2: {:0.2f}".format(xput_mw2))
print("Queue Length MW1: {:0.2f}".format(reqs_mw1['queueLength'].mean()))
print("Queue Length MW2: {:0.2f}".format(reqs_mw2['queueLength'].mean()))
print("Total Throughput MT: {:0.2f}".format(mt_averages.loc['mean', 'throughput']))
print("Avg Response Time MT: {:0.4f}".format(mt_averages.loc['mean', 'responsetime']))
print("Utilization MW1 NetThread: {:0.2f}".format(xput_mw1 / 1000 * reqs_mw1['netthreadServiceTime_ms'].mean() / num_workers))
print("Utilization MW2 NetThread: {:0.2f}".format(xput_mw2 / 1000 * reqs_mw2['netthreadServiceTime_ms'].mean() / num_workers))
print("Utilization MW1 workers: {:0.2f}".format(xput_mw1 / 1000 * reqs_mw1['workerServiceTime_ms'].mean() / num_workers))
print("Utilization MW2 workers: {:0.2f}".format(xput_mw2 / 1000 * reqs_mw2['workerServiceTime_ms'].mean() / num_workers))

print("Utilization MW1 memcached: {:0.2f}".format(xput_mw1 / 1000 * reqs_mw1['memcachedRTT_ms'].mean() / num_workers))
print("Utilization MW2 memcached: {:0.2f}".format(xput_mw2 / 1000 * reqs_mw2['memcachedRTT_ms'].mean() / num_workers))

Service times input for simple model:
1: 1.2930
2: 1.0774
3: 0.0048
4: 0.0040
5: 2.5422
6: 2.3935
7: 1.2930
8: 1.0774



Actually measured metrics: (compare with model output)
Throughput MW1: 3097.04
Throughput MW2: 3300.54
Queue Length MW1: 112.98
Queue Length MW2: 111.77
Total Throughput MT: 6420.49
Avg Response Time MT: 39.6781
Utilization MW1 NetThread: 0.00
Utilization MW2 NetThread: 0.00
Utilization MW1 workers: 0.98
Utilization MW2 workers: 0.99
Utilization MW1 memcached: 0.98
Utilization MW2 memcached: 0.98


In [7]:
pd.concat([reqs_mw1, reqs_mw2])['workerServiceTime_ms'].mean()

2.4655071394019856